In [2]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd
import datetime as dt

from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from scipy import stats

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Solar_Power"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [3]:
#import data
plant_1_gen = pd.read_csv("Plant_1_Generation_data.csv")
plant_2_gen = pd.read_csv("Plant_2_Generation_data.csv")
plant_1_wea = pd.read_csv("Plant_1_Weather_Sensor_data.csv")
plant_2_wea = pd.read_csv("Plant_2_Weather_Sensor_data.csv")


## Data Exploration
our dataset consist of 4 files 2 for each plant
- power generation data
- weather sensor data

### The power generations data consists of 7 fields:
| DATE_TIME | PLANT_ID | SOURCE_KEY | DC_POWER | AC_POWER | DAILY_YIELD | TOTAL_YIELD |
| --------- | -------- | ---------- | -------- | -------- | ----------- | ----------- |
| Date and time for each observation (%d-%m-%Y %H:%M). Observations recorded at 15 minute intervals. | Plant ID - this will be common for the entire file. | Source key in this file stands for the inverter id. | Amount of DC power generated by the inverter (source_key) in this 15 minute interval. Units - kW. | Amount of AC power generated by the inverter (source_key) in this 15 minute interval. Units - kW. | Daily yield is a cumulative sum of power generated on that day till that point in time. | This is the total yield for the inverter till that point in time. |

### The weather data consists of 6 fields:
| DATE_TIME | PLANT_ID | SOURCE_KEY | AMBIENT_TEMPERATURE | MODULE_TEMPERATURE | IRRADIATION |
| --------- | -------- | ---------- | -------- | -------- | ----------- |
| Date and time for each observation (%d-%m-%Y %H:%M). Observations recorded at 15 minute intervals. | Plant ID - this will be common for the entire file. | Stands for the sensor panel id. This will be common for the entire file because there's only one sensor panel for the plant. | This is the ambient temperature at the plant. | There's a module (solar panel) attached to the sensor panel. This is the temperature reading for that module. | Amount of irradiation for the 15 minute interval. Solar irradiation is the quantity that measures the energy per unit area of ​​incident solar radiation on a surface - the power received during a time (J/m2 or Wh/m2 )  |

Data is gathered every 15 minutes for 34 days.

Expecting 3264 samples per sampling point:

24 hours x 4 times per hour x 34 days = 3264 samples

## check columns for missing values
### plant 1

In [13]:
missing1 = plant_1_gen.copy()


= plant_1_gen.groupby("SOURCE_KEY")["DATE_TIME"].count()
plant_1_gen[plant_1_gen["SOURCE_KEY"]=="bvBOhCH3iADSZry"]
if plant_1_gen.index.name != "SOURCE_KEY":
    plant_1_gen.set_index(plant_1_gen["SOURCE_KEY"],inplace=True)
plant_1_gen
# print(plant1_info)

ValueError: 'SOURCE_KEY' is both an index level and a column label, which is ambiguous.

In [4]:
plant1_wea = plant_1_wea.isnull().sum()
print(plant1_wea)

DATE_TIME              0
PLANT_ID               0
SOURCE_KEY             0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
dtype: int64


### Plant 2

In [5]:

plant2_info = plant_2_gen.isnull().sum()
print(plant2_info)

DATE_TIME      0
PLANT_ID       0
SOURCE_KEY     0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
TOTAL_YIELD    0
dtype: int64


In [6]:
plant2_wea = plant_2_wea.isnull().sum()
print(plant2_wea)

DATE_TIME              0
PLANT_ID               0
SOURCE_KEY             0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
dtype: int64


## How many sensors are at each plant?

In [7]:
print("Plant 1 has",plant_1_gen['SOURCE_KEY'].nunique(),"sensors")
print("Plant 2 has",plant_2_gen['SOURCE_KEY'].nunique(),"sensors")

Plant 1 has 22 sensors
Plant 2 has 22 sensors


### Do each of the sensors have continues time series?

In [14]:
plant_1_gen["DATE_TIME"] = pd.to_datetime(plant_1_gen["DATE_TIME"])
plant_1_gen["Human_key"] = plant_1_gen["SOURCE_KEY"].map({val: f"S{i+1:02d}" for i, val in enumerate(plant_1_gen["SOURCE_KEY"].unique())})
plant_1_sensor_1 = plant_1_gen[(plant_1_gen["Human_key"] == "S01")]
plant_1_sensor_1.set_index(pd.DatetimeIndex(plant_1_sensor_1["DATE_TIME"]),inplace=True)
plant_1_sensor_1["AC_POWER"].resample('15 min').min()

DATE_TIME
2020-01-06 00:00:00    0.0
2020-01-06 00:15:00    0.0
2020-01-06 00:30:00    0.0
2020-01-06 00:45:00    0.0
2020-01-06 01:00:00    0.0
                      ... 
2020-12-06 22:45:00    0.0
2020-12-06 23:00:00    0.0
2020-12-06 23:15:00    0.0
2020-12-06 23:30:00    0.0
2020-12-06 23:45:00    0.0
Freq: 15T, Name: AC_POWER, Length: 32256, dtype: float64

In [ ]:
#Data for 34 days and continuous dates
# TODO : refine for each sensor
plant_1_gen["DATE_TIME"] = pd.to_datetime(plant_1_gen["DATE_TIME"])
plant_1_gen["DATE"] = plant_1_gen["DATE_TIME"].dt.date
# fig, axs = plt.subplots(nrows=11, ncols=2, figsize=(15,12))
# plt.subplots_adjust(hspace=0.5)
# fig.suptitle("date", fontsize = 18, y=0.95)
# plt.show()
for title, group in plant_1_gen[["SOURCE_KEY","DATE_TIME","AC_POWER"]].groupby(plant_1_gen["DATE_TIME"].dt.date):
    for title2, group2 in group[["SOURCE_KEY","DATE_TIME","AC_POWER"]].groupby("SOURCE_KEY"):
        group2.plot(x="DATE_TIME",y="AC_POWER",title=f"{title}-{title2}")

In [ ]:
#Inverterids
#Some inverters have more data points then the others
print("Plant 1 statistics \n", plant_1_gen.SOURCE_KEY.value_counts())
print("Plant 2 statistics \n", plant_2_gen.SOURCE_KEY.value_counts())

In [ ]:
#Plant 1 weather data
plant_1_wea['DATE_TIME'] = pd.to_datetime(plant_1_wea['DATE_TIME']) 
plant_1_wea['TIME'] = plant_1_wea['DATE_TIME'].dt.time 
#convert datetime column to just date
plant_1_wea['DATE'] = pd.to_datetime(plant_1_wea['DATE_TIME'].dt.date)
print(plant_1_wea['DATE'])

#Plant 2 weather data
plant_2_wea['DATE_TIME'] = pd.to_datetime(plant_2_wea['DATE_TIME']) 
plant_2_wea['TIME'] = plant_2_wea['DATE_TIME'].dt.time 
#convert datetime column to just date
plant_2_wea['DATE'] = pd.to_datetime(plant_2_wea['DATE_TIME'].dt.date)


Analysis of weather dataset for both the plants:
Observations:
1. The mean solar irradiation values for both plants are similar.
2. The mean module temperature of Plant 1 is  lower than Plant 2 most of the time.
3. The mean ambient temperature of Plant 1 is much lower than Plant 2.


In [ ]:
# Comparing both plants
# Daily Irradiation
ambient_compare = sns.lineplot(x='DATE', y='IRRADIATION', data=plant_1_wea, err_style='band', label='Plant 1')
sns.lineplot(x='DATE', y='IRRADIATION', data=plant_2_wea, err_style='band', label='Plant 2', ax=ambient_compare)
plt.ylabel('Irradiation')
plt.xlabel('Date')
plt.title('Daily Solar Irradiation for Both Plants')
plt.xticks(rotation=45)
plt.show()

# Daily Module Temperature
modtemp_compare = sns.lineplot(x='DATE', y='MODULE_TEMPERATURE', data=plant_1_wea, err_style='band', label='Plant 1')
sns.lineplot(x='DATE', y='MODULE_TEMPERATURE', data=plant_2_wea, err_style='band', label='Plant 2', ax=modtemp_compare)
plt.ylabel('Module Temperature')
plt.xlabel('Date')
plt.title('Daily Module Temperature for Both Plants')
plt.xticks(rotation=45)
plt.show()

# Daily Ambient Temperature
ambtemp_compare = sns.lineplot(x='DATE', y='AMBIENT_TEMPERATURE', data=plant_1_wea, err_style='band', label='Plant 1')
sns.lineplot(x='DATE', y='AMBIENT_TEMPERATURE', data=plant_2_wea, err_style='band', label='Plant 2', ax=ambtemp_compare)
plt.ylabel('Ambient Temperature')
plt.xlabel('Date')
plt.title('Daily Ambient Temperature for Both Plants')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# The mean of solar Irradiation for both plants are similar
mean_irradiationplant1 = plant_1_wea['IRRADIATION'].mean()
print('Mean of solar irradition from Plant1', mean_irradiationplant1)
mean_irradiationplant2 =  plant_2_wea['IRRADIATION'].mean()
print('Mean of solar irradition from Plant2', mean_irradiationplant2)

In [ ]:
# The mean of Module Temperature for both plants (Plant 1 is lower then Plant 2)
mean_moduletempplant1 = plant_1_wea['MODULE_TEMPERATURE'].mean()
print('Mean of Module Temperature from Plant1', mean_moduletempplant1)
mean_moduletempplant2 =  plant_2_wea['MODULE_TEMPERATURE'].mean()
print('Mean of Module Temperature from Plant2', mean_moduletempplant2)

In [ ]:
# The mean of Ambient Temperature for both plants (Plant 1 is lower then Plant 2)
mean_ambienttempplant1 = plant_1_wea['AMBIENT_TEMPERATURE'].mean()
print('Mean of Module Temperature from Plant1', mean_ambienttempplant1)
mean_ambienttempplant2 =  plant_2_wea['AMBIENT_TEMPERATURE'].mean()
print('Mean of Module Temperature from Plant2', mean_ambienttempplant2)

In [ ]:
# drop unwanted columns. 
df_weather1 = plant_1_wea.drop(['PLANT_ID', 'SOURCE_KEY'], axis=1)
df_plant1 = plant_1_gen.drop(['PLANT_ID'], axis=1)

In [ ]:
#Formatedattime
df_plant1['DATE_TIME']= pd.to_datetime(df_plant1['DATE_TIME'],format='%d-%m-%Y %H:%M')
df_weather1['DATE_TIME']= pd.to_datetime(df_weather1['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')
df_plant1.head()

In [ ]:
#Merge the dataset
df_plant_weather1 = df_plant1.merge(df_weather1, left_on='DATE_TIME', right_on='DATE_TIME')
df_plant_weather1.head()


######Data Correlation Analysis########

Observations:
1. High correlation between DC Power and AC power generation
2. High correlation between DC Power and IRRADIATION
3. Strong correlation between DC Power, AC Power and Module Temperature and Ambient Temperature

In [ ]:
corr_matrix = df_plant_weather1.corr()

In [ ]:
corr_matrix["DC_POWER"].sort_values(ascending=False)

In [ ]:
# from pandas.tools.plotting import scatter_matrix # For older versions of Pandas
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Data_analysis"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


attributes = ["DC_POWER", "AC_POWER", "IRRADIATION",
              "MODULE_TEMPERATURE","AMBIENT_TEMPERATURE"]
scatter_matrix(df_plant_weather1[attributes], figsize=(20, 12))
save_fig("scatter_matrix_plot")

Analysis of DC power generated from each Source Keys.
1. The DC power generation plot shows multiple occasion where power generated was zero during daytime.
2. Distribution plot of solar irradiation exhibits that the solar radiation never dropped to a lower value at day time. 
3. Analysis shows some inverters received no DC power even through there was enough sunlight
4. It could be concluded that the DC power generated and solar irradiation has a linear relationship.

In [ ]:
#DC power generated from each source keys
sources=df_plant_weather1.copy()
sources['time']=sources['DATE_TIME'].dt.time
sources.set_index('time').groupby('SOURCE_KEY')['DC_POWER'].plot(style='o',legend=True,figsize=(20,10))
plt.title('DC Power generated for all interverters(source_keys)',size=17)
plt.ylabel('DC POWER ( kW )',color='navy',fontsize=17)
plt.show()

In [ ]:
#Solar IRRADIATION generated for days
sources=df_plant_weather1.copy()
sources['time']=sources['DATE_TIME'].dt.time
sources['date']=sources['DATE_TIME'].dt.date
sources.set_index('time').groupby('date')['IRRADIATION'].plot(style='g',legend=True,figsize=(20,10))
plt.title('Distribution of IRRADIATION during daytime',size=17)
plt.ylabel('IRRADIATION',color='navy',fontsize=17)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
fig_irr = sns.scatterplot(data=df_plant_weather1, x="IRRADIATION", y="DC_POWER", hue="SOURCE_KEY", palette="tab20")
plt.show()

## Resample data

dagens dato = 
gårsdagens = dagens - 1 dag

ac_predict = df.apply( func (source_key == source_key) and (date === date -1) null

fill_na(0)

In [ ]:
def make_sample(df,min):
    x = df.copy()
    x.drop("SOURCE_KEY", axis = 1)
    x.set_index("DATE_TIME", inplace = True)
    x.resample(str(min) + " min").mean()
    return  x.fillna(0)

#print value at 20th may
def show15may(df):
    start_time =pd.to_datetime('15/5/2020')
    end_time =pd.to_datetime('20/5/2020')    

dagens Dat

persistant = df["AC_POWER" DATE_TIME == td.datediff ('d',-1)]

df15 = make_sample(df_plant_weather1, 15)
show15may(df15)
df30 = make_sample(df_plant_weather1, 30)
show15may(df30)
df60 = make_sample(df_plant_weather1, 60)
show15may(df60)
df2h = make_sample(df_plant_weather1, 120)
show15may(df2h)
df4h = make_sample(df_plant_weather1, 240)
show15may(df4h)

In [ ]:
def show_null_ac(df):
    print(df[df["AC_POWER"].isnull()].sum())

show_null_ac(df15)
show_null_ac(df30) 
show_null_ac(df60) 
show_null_ac(df2h)    
show_null_ac(df4h)

## Split data

In [ ]:


X1=df_plant_weather1[['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE']] # Features 
y1=df_plant_weather1['DC_POWER'] # independent var / predictor

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.20, random_state=1)

print("Shape of each Dataset : ")
print("X Train Shape = ",X1_train.shape)
print("Y Train Shape = ",y1_train.shape)
print("X Test Shape  = ",X1_test.shape)
print("Y Test Shape  = ",y1_test.shape)

In [ ]:

def conv_pct(score):
    return round(score*100,0)

def linear_pipeline(df):
    lm = LinearRegression()
    a = df[['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE']] # Features
    b = df['DC_POWER'] # Target
    a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=0.20, shuffle=False)
    lm.fit(a_train, b_train)
    training_prediction = lm.predict(a_train)
    testing_prediction = lm.predict(a_test)
    train_MAE = metrics.mean_absolute_error(b_train, training_prediction)
    train_MSE = metrics.mean_squared_error(b_train, training_prediction)
    train_RMSE = np.sqrt(metrics.mean_squared_error(b_train, training_prediction))
    test_MAE = metrics.mean_absolute_error(b_test, testing_prediction)
    test_MSE = metrics.mean_squared_error(b_test, testing_prediction)
    test_RMSE = np.sqrt(metrics.mean_squared_error(b_test, testing_prediction))
    train_acc_pct = conv_pct(lm.score(a_train, b_train))
    test_acc_pct = conv_pct(lm.score(a_test, b_test))
    return [training_prediction,testing_prediction]

linear_result15 = linear_pipeline(df15)
linear_result30 = linear_pipeline(df30)
linear_result60 = linear_pipeline(df60)
linear_result2h = linear_pipeline(df2h)


print(linear_pipeline(df15))
print(linear_pipeline(df30))
print(linear_pipeline(df60))
print(linear_pipeline(df2h))


In [ ]:
 #fit on training data
lm.fit(X1_train, y1_train)
lm.intercept_, lm.coef_

In [ ]:
print ('Plant 1 linear regression cofficients are:', X.columns, lm.coef_ )
print('Plant 1 intercept', lm.intercept_)

In [ ]:
#Prediction from model
Prediction_plant1 = lm.predict(X1_test)
Prediction_plant1
Y1_pred_train = lm.predict(X1_train)
Y1_pred_test = lm.predict(X1_test)

print ('Prediction Train dataset', Y1_pred_train)
print ('Prediction Train dataset', Y1_pred_test)

In [ ]:

# This also includes faulty days dataset
plt.scatter(y1_test, Y1_pred_test, color="b", label="Predicted Output")
#plt.scatter(Y_test, DC_POWER, label="Measured Output")
#plt.plot(X, Y, "b.")
plt.legend()
plt.title('Actual Solar Output Values vs Predicted Values for Plant 1')
plt.xlabel('Predicted Output')
plt.ylabel('Actual Output')

In [ ]:


# PLANT 1
MAE1 = metrics.mean_absolute_error(y1_test, Y1_pred_test)
MSE1 = metrics.mean_squared_error(y1_test, Y1_pred_test)
RMSE1 = np.sqrt(metrics.mean_squared_error(y1_test, Y1_pred_test))
print('Metrics for Plant 1 Linear Model')
print('MAE: ', MAE1)
print('MSE: ',MSE1)
print('RMSE: ', RMSE1)
print()

In [ ]:
print("Model Evaluation for Linear Regression Model 1 using RMSE")
print("")
RMSE_train_1 = np.sqrt( metrics.mean_squared_error(y1_train, Y1_pred_train))
RMSE_test_1 = np.sqrt(metrics.mean_squared_error(y1_test, Y1_pred_test))
print('RMSE for training set = {}'.format(round(RMSE_train_1,2)))
print('RMSE for test set = {}'.format(round(RMSE_test_1,2)))

In [ ]:
## acuracy score
train_score_1 = lm.score(X1_train, y1_train)
test_score_1 = lm.score(X1_test, y1_test)

print("Model 1 accuracy score: ")
print("")
print("Train Score = ",round(train_score_1*100,0),"%")
print("Test Score  = ",round(test_score_1*100,0), "%")

Model 2: Random Forest

In [ ]:
## Select columns that will be used to create train and test dataset
model_2 = df_plant_weather1
model_2.head()

In [ ]:
## create list using feature columns
#model_2_features = model_2['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']

# use above list to select subset of original dataframe
#X2 = model_2[model_2_features]
#y3 = model_2["DC_POWER"] # dependent variable / output

X2 = model_2[['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']] # Features
y2 = model_2['DC_POWER'] # Target

X2_train, X2_test, y2_train, y2_test=train_test_split(X2, y2, test_size=0.20, random_state=1) 

print("Shape of each New Dataset : ")
print("X Train Shape = ",X2_train.shape)
print("Y Train Shape = ",y2_train.shape)
print("X Test Shape  = ",X2_test.shape)
print("Y Test Shape  = ",y2_test.shape)

In [ ]:

RF = RandomForestRegressor()
RF.fit(X2_train, y2_train)

In [ ]:
y2_pred_train = RF.predict(X2_train)
y2_pred_test = RF.predict(X2_test)

In [ ]:
print("Model Evaluation for Random Forest Model 2 using RMSE")

RMSE_train_2 = np.sqrt( metrics.mean_squared_error(y2_train, y2_pred_train))
RMSE_test_2 = np.sqrt(metrics.mean_squared_error(y2_test, y2_pred_test))
MSE_train_2 = ( metrics.mean_squared_error(y2_train, y2_pred_train))
print('RMSE for training set is {}'.format(round(RMSE_train_2,2)))
print('RMSE for test set is {}'.format(round(RMSE_test_2,2)))
print('MSE for train set is {}'.format(round(MSE_train_2,2)))

In [ ]:
## acuracy scorefor Model 2
## acuracy score
train_score_2 = RF.score(X2_train, y2_train)
test_score_2 = RF.score(X2_test, y2_test)

print("Model 2 accuracy score: ")
print("")
print("Train Score = ",round(train_score_2*100,0),"%")
print("Test Score  = ",round(test_score_2*100,0), "%")

In [ ]:
# This also includes faulty days dataset
plt.scatter(y2_test, y2_pred_test,color="r", label="Predicted Output test")
plt.scatter(y2_train, y2_pred_train,color="b", label="Predicted Output train")
#plt.scatter(Y_test, DC_POWER, label="Measured Output")
#plt.plot(X, Y, "b.")
plt.legend()
plt.title('Actual Solar Output Values vs Predicted Values for Plant 1 using Random Forest')
plt.xlabel('Predicted Output')
plt.ylabel('Actual Output')

In [ ]:
#Feature Scaling

sc = StandardScaler()
X_train = sc.fit_transform(X2_train)
X_test = sc.transform(X2_test)

In [ ]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X2_train, y2_train)
y3_pred = regressor.predict(X2_test)

In [ ]:

conv_2020 = df_plant_weather1
conv_2020.head()

In [ ]:
X = conv_2020['DATE_TIME']
y = conv_2020['DC_POWER']
slope, intercept, r, p, std_err = stats.linregress(X, y)

def modelPrediction(x):
  return slope * x + intercept

label = 'Model Prediction for conventional avocado in 2020 is = '

model = list(map(modelPrediction, X)) # scipy

x_pred = 2020
y_pred = modelPrediction(x_pred)
print(label)

ap_conv_2020 = round(y_pred, 2)
print('$ {}'.format(ap_conv_2020))
print("")

plt.scatter(X, y) # Scatter Plot
plt.plot(X, model, color='red')
plt.ylim(ymin=0) # starts at zero
plt.legend(['Model Prediction', 'Conventional Avocado Prices (2015-2019)'])
plt.show()